In [ ]:
  # mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.7 MB/s eta 0:00:00


In [ ]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import json
import random
import pandas as pd
import csv
import os

## Prep dataset

In [ ]:
df = pd.read_json("hf://datasets/FreedomIntelligence/medical-o1-verifiable-problem/medical_o1_verifiable_problem.json")
df.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


In [ ]:
df.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


## Set up model definition

In [ ]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the Llama model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    token=userdata.get('HF_TOKEN'))

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

1

## Eval of Llama

In [ ]:
x = df.loc[0, "Open-ended Verifiable Question"]
y_star = df.loc[0, "Ground-True Answer"]

In [ ]:
print('x:', x)
print('y_star:', y_star)

x: An 88-year-old woman with osteoarthritis is experiencing mild epigastric discomfort and has vomited material resembling coffee grounds multiple times. Considering her use of naproxen, what is the most likely cause of her gastrointestinal blood loss?
y_star: Gastric ulcer


In [ ]:
init_prompt = f"""<question>
{x}
</question>
Please respond to the above question <question> using the Chain of Thought (CoT) reasoning method.
Your response should consist of multiple steps, each of which includes three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
- **’Inner Thinking’**: This is the step where thinking is done. Note that multiple ’Inner Thinking’ steps are
required to describe thorough reasoning. Each step should first generate a brief title.
- **’Final Conclusion’**: At this stage, you summarize the correct reasoning from previous ’Inner Thinking’
steps and provide the final answer. No title is required here.
- **’Verification’**: At this stage, you verify the conclusion from the "Final Conclusion" step. If the
conclusion holds, end the process. If not, return to "Inner Thinking" for further reasoning. No title is required
here.
The output format must strictly follow the JSON structure
"""

In [ ]:
chat_formatted = tokenizer.apply_chat_template(
    [{"role": "user", "content": init_prompt}],
    tokenize=False,
    add_generation_prompt=True
)

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
inputs = tokenizer.apply_chat_template(
	init_prompt,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
verif_prompt = f"""<Model Response>
{response}
</Model Response>
<Reference Answer>
{y_star}
</Reference Answer>
You are provided with a model-generated response (<Model Response>) and a reference answer (<Reference
Answer>). Compare the model response with the reference answer and determine its correctness. Your task
is to simply output "True" if the response is correct, and "False" otherwise"""

In [ ]:
outputs = pipe(
        verif_prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )
response = outputs[0]["generated_text"]
print(response)

### Defining Prompt Templates

In [ ]:
init_prompt = """<question>
{x}
</question>
Please respond to the above question <question> using the Chain of Thought (CoT) reasoning method.
Your response should consist of multiple steps, each of which includes three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
- **’Inner Thinking’**: This is the step where thinking is done. Note that multiple ’Inner Thinking’ steps are
required to describe thorough reasoning. Each step should first generate a brief title.
- **’Final Conclusion’**: At this stage, you summarize the correct reasoning from previous ’Inner Thinking’
steps and provide the final answer. No title is required here.
- **’Verification’**: At this stage, you verify the conclusion from the "Final Conclusion" step. If the
conclusion holds, end the process. If not, return to "Inner Thinking" for further reasoning. No title is required
here.
The output format must strictly follow the JSON structure
"""

In [ ]:
verif_prompt = """<Model Response>
{response}
</Model Response>
<Reference Answer>
{y_star}
</Reference Answer>
You are provided with a model-generated response (<Model Response>) and a reference answer (<Reference
Answer>). Compare the model response with the reference answer and determine its correctness. Your task
is to simply output "True" if the response is correct, and "False" otherwise"""

Exploring New Paths

In [ ]:
exploring_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section.
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning by exploring new approaches to solving this problem and construct a new Final
Conclusion.
"""

Backtracking

In [ ]:
backtracking_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section.
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning using **backtracking** to revisit earlier points of reasoning and construct a new Final
Conclusion.
"""

Verifications

In [ ]:
verif_search_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section.
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning by making precise **corrections** to address prior flaws and construct a new Final
Conclusion.
"""

Corrections

In [ ]:
corr_prompt = """<question>
{x}
</question>
<previous reasoning>
{response}
<previous reasoning>
<response requirements>
Your response must include the following steps, each composed of three types of actions: **"Inner
Thinking"**, **"Final Conclusion"**, and **"Verification"**:
1. **Inner Thinking**: Break down the reasoning process into multiple concise steps. Each step should start
with a brief title to clarify its purpose.
2. **Final Conclusion**: Summarize the correct reasoning from all previous ’Inner Thinking’ steps and
provide the final answer. No title is needed for this section
3. **Verification**: Verify the accuracy of the "Final Conclusion". If it holds, conclude the process.
Otherwise, return to "Inner Thinking" for further refinement.
</response requirements>
<question> represents the question to be answered, and <previous reasoning> contains your prior reasoning.
Your task is to continue from the current ’Verification’ step. I have manually reviewed the reasoning and
determined that the **Final Conclusion** is false. Your ’Verification’ results must align with mine. Proceed
to refine the reasoning by conducting a thorough **validation** process to ensure validity and construct a
new Final Conclusion.
"""

## Function Form

In [ ]:
def clean_json_output(raw_text):
    return raw_text.strip().removeprefix("```json").removeprefix("```").removesuffix("```").strip()

In [ ]:
def run_huatuo(prompt):

  outputs = pipe(
        prompt,
        max_new_tokens=1024,
        temperature=1,
        return_full_text=False
    )

  response = outputs[0]["generated_text"]

  return response

In [ ]:
def dsft_approach(x, y_star):

  prompts = []

  prompt = init_prompt.format(x=x)
  prompts.append(prompt)
  response = run_huatuo(prompt)

  v_prompt = verif_prompt.format(response=response, y_star=y_star)
  verif_result = run_huatuo(v_prompt)

  clean_res = False

  if 'True' in verif_result or 'true' in verif_result:
    clean_res = True

  if clean_res == False:

    result = False
    idx = 0

    searching_prompts = [exploring_prompt, backtracking_prompt, verif_search_prompt, corr_prompt]
    random.shuffle(searching_prompts)

    searching_prompts = searching_prompts[:3]

    while result == False and idx <= 2:

      prompt = searching_prompts[idx].format(x=x, response=response)
      prompts.append(prompt)
      response = run_huatuo(prompt)

      v_prompt = verif_prompt.format(response=response, y_star=y_star)
      verif_result = run_huatuo(v_prompt)

      if 'True' in verif_result or 'true' in verif_result:
        clean_res = True
        result = True
      else:
        idx += 1

  return response, prompts, clean_res

In [ ]:
results = []

csv_path = '/content/drive/MyDrive/MIDS/dsft_huatuo_results.csv'
fieldnames = ['Answer', 'Prompts', 'post_ft_results']

if not os.path.exists(csv_path):
    with open(csv_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

for i in range(300):

  x = df.loc[i, "Open-ended Verifiable Question"]
  y_star = df.loc[i, "Ground-True Answer"]

  answer, prompts, res = dsft_approach(x, y_star)

  with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([answer, prompts, res])

  results.append({
        "Answer": answer,
        "Prompts": prompts,
        "post_ft_results": res
    })

In [ ]:
# results_df = pd.DataFrame(results)

# results_df['num_prompts'] = results_df['Prompts'].apply(len)
# results_df["baseline_results"] = (results_df['num_prompts'] == 1).astype(int)
# results_df['post_ft_results'] = results_df['post_ft_results'].astype(int)

# results_df.head(10)